In [1]:
#boto3 => Pyhton library for calling up AWS services
import boto3
import sagemaker
from sagemaker import get_execution_role

upload the datasets into S3 bucket

In [2]:
#provide the name and location of the files to be stored in the S3 bucket
bucket_name = 's3loanprediction'
train_file_name = 'Loan Prediction/Train_final.csv'
val_file_name = 'Loan Prediction/Val_final.csv'
test_file_name = 'Loan Prediction/Test_final.csv'

model_output_location = r's3://{0}/LoanPrediction/model'.format(bucket_name)
train_file_location = r's3://{0}/{1}'.format(bucket_name, train_file_name)
val_file_location = r's3://{0}/{1}'.format(bucket_name, val_file_name)
test_file_location = r's3://{0}/{1}'.format(bucket_name, test_file_name)

In [3]:
#define a method for writing into s3 bucket
def write_to_s3(filename, bucket, key):
    with open(filename, 'rb') as f:
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [4]:
write_to_s3('Train_final.csv', bucket_name, train_file_name)
write_to_s3('Val_final.csv', bucket_name, val_file_name)
write_to_s3('Test_final.csv', bucket_name, test_file_name)

## Training Algorithm Docker Image
### SageMaker maintains a separate image for algorithm and region
https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

In [5]:
# Establish a session with AWS
sess = sagemaker.Session()

In [6]:
role = get_execution_role()

In [7]:
# This role contains the permissions needed to train, deploy models
# SageMaker Service is trusted to assume this role
print(role)

arn:aws:iam::267391731566:role/service-role/AmazonSageMaker-ExecutionRole-20200909T224279


In [8]:
# Sagemaker API now maintains the algorithm container mapping for us
# Specify the region, algorithm and version
container = sagemaker.amazon.amazon_estimator.get_image_uri(
    sess.boto_region_name,
    "xgboost", 
    "latest")

# container = {'us-east-1': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'}

print('Using SageMaker XGBoost container:\n{} ({})'.format(container, sess.boto_region_name))

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Using SageMaker XGBoost container:
811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest (us-east-1)


In [9]:
print(boto3.Session().region_name)

us-east-1


## Build Model

In [10]:
# Configure the training job
# Specify type and number of instances to use
# S3 location where final artifacts needs to be stored

#   Reference: http://sagemaker.readthedocs.io/en/latest/estimators.html

estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.m4.xlarge',
    output_path=model_output_location,
    sagemaker_session=sess,
    base_job_name ='xgboost-loanprediction')

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [11]:
#setting hyperparameters corresponding to the XGBoost algorithm
estimator.set_hyperparameters(max_depth=5, 
                              objective = 'binary:logistic', 
                              eta=0.1,
                              subsample=0.7,
                              num_round=10,
                              eval_metric = 'auc')

In [12]:
estimator.hyperparameters()

{'max_depth': 5,
 'objective': 'binary:logistic',
 'eta': 0.1,
 'subsample': 0.7,
 'num_round': 10,
 'eval_metric': 'auc'}

In [13]:
#training the model using fit model
training_file = sagemaker.session.s3_input(s3_data=train_file_location, content_type = "csv")
validation_file = sagemaker.session.s3_input(s3_data=val_file_location, content_type = "csv")

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [14]:
print(training_file.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://s3loanprediction/Loan Prediction/Train_final.csv', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'csv'}


In [15]:
print(validation_file.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://s3loanprediction/Loan Prediction/Val_final.csv', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'csv'}


In [16]:
data_channels = {'train':training_file, 'validation':validation_file}

### Train the model

In [17]:
# XGBoost supports "train", "validation" channels
# Reference: Supported channels by algorithm
#   https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html
estimator.fit(inputs=data_channels, logs=True)

2021-04-12 13:15:44 Starting - Starting the training job...
2021-04-12 13:15:46 Starting - Launching requested ML instancesProfilerReport-1618233344: InProgress
......
2021-04-12 13:17:03 Starting - Preparing the instances for training.........
2021-04-12 13:18:40 Downloading - Downloading input data...
2021-04-12 13:19:00 Training - Downloading the training image..Arguments: train
[2021-04-12:13:19:25:INFO] Running standalone xgboost training.
[2021-04-12:13:19:25:INFO] File size need to be processed in the node: 0.05mb. Available memory size in the node: 8407.61mb
[2021-04-12:13:19:25:INFO] Determined delimiter of CSV input is ','
[13:19:25] S3DistributionType set as FullyReplicated
[13:19:25] 460x11 matrix with 5060 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-04-12:13:19:25:INFO] Determined delimiter of CSV input is ','
[13:19:25] S3DistributionType set as FullyReplicated
[13:19:25] 116x11 matrix with 1276 entries loaded from /opt/ml/inpu

### Deploying the Model

In [18]:
#deploying the model and create an end point
predictor = estimator.deploy(initial_instance_count = 1,
                             instance_type = 'ml.m4.xlarge',
                             endpoint_name = 'xgboost-loanprediction-ver1')

-----------------!